# Finding the nearest train/tube station

The purpose of this project is to create module that takes a London postcode input e.g. N4 4AF and will output the nearest train station to it.

In [28]:
import pandas as pd
import requests
from geopy import distance

In [29]:
# A csv file of london stations can be downloaded from this link
# https://www.doogal.co.uk/london_stations.php

# TODO join station data to tube line
gb_stations_df = pd.read_csv('gb_stations.csv')

In [30]:
GB_STATIONS = []
for x in gb_stations_df[['Station', 'Latitude', 'Longitude']].values:
    GB_STATIONS.append((x[0],(x[1],x[2])))

In [31]:
gb_stations_df.head(2)

,Station,Postcode,Latitude,Longitude,TLC,NLC,Owner,Entries and exits 2020,Entries and exits 2019,Entries and exits 2018,...,Interchanges 2014,Interchanges 2013,Interchanges 2012,Interchanges 2011,Interchanges 2010,Interchanges 2009,Interchanges 2008,Interchanges 2007,Interchanges 2006,Interchanges 2005
0,Abbey Wood,SE2 9RH,51.491061,0.121394,ABW,5131,TfL Rail,3825206,3769402.0,3124856.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aber,CF83 1AQ,51.574969,-3.229829,ABE,3813,TfW Rail,228480,245218.0,251108.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
GB_STATIONS[0][1]

(51.491060562701904, 0.12139402536664698)

In [5]:
# test_postcodes = ['N15 3AD', 'N4 4AF', 'N10 3QS', 'N15 4AR', 'N15 4JF']

In [53]:
# TODO refactor - use less code
# TODO Add documentation
# TODO experiment using getters and setters for km?
class NearestStation:
        
    def __init__(self, postcode, unit='m'):
        self.postcode = postcode.lower()
        self.lat_lon = self.geo_dict['lat_lon']
        self.unit = unit
        self.nearest_station = self.find_nearest_station() #nearest_m TODO
        #TODO add nearest in km
    
#     @property
#     def postcode(self):
#         return self._postcode.lower()
    
#     @postcode.setter
#     def postcode(self, new_postcode):
#         if len(new_postcode) <= 7:
#             self._postcode = new_postcode.lower()
#         else:
#             raise ValueError('Enter a suitable postcode')
        
    @property
    def unit(self):
        return self._unit
    
    @unit.setter
    def unit(self, new_unit):
        if new_unit in ['m', 'km']:
            self._unit = new_unit
        else:
            raise ValueError('Enter "m" for miles or "km" for kilometers')
    
#     setter for 
        
    @property
    def endpoint(self):
        pc = self.postcode.replace(' ','')
        return f'http://api.postcodes.io/postcodes/{pc}'
    
    @property
    def geo_dict(self):
        geocoded_dict = {}
        r = requests.get(self.endpoint)
        geocoded_dict['postcode'] = r.json()['result']['postcode']
        geocoded_dict['lat_lon'] = (r.json()['result']['latitude'], r.json()['result']['longitude'])
        return geocoded_dict
    
    def distance_miles(self, x):
        return round(distance.distance(self.lat_lon, x).miles,1)
    
    def distance_km(self, x):
        return round(distance.distance(self.lat_lon, x).km,1)
    
    def find_nearest_station(self): 
        #TODO give user option to enter their own list of tuples with locations - new function?
        if self.unit == 'm':
            # could split the below into another function to make clearer - use of HIGHER ORDER FUNCTIONS
            distances = [(x[0], round(distance.distance(self.lat_lon, x[1]).miles,1)) for x in GB_STATIONS]
            distances.sort(key=lambda x:x[1])
            return (distances[0])
        elif self.unit == 'km':
            distances = [(x[0], round(distance.distance(self.lat_lon, x[1]).km,1)) for x in GB_STATIONS]
            distances.sort(key=lambda x:x[1])
            return (distances[0])
        
#     @property
#     def nearest_station(self):
#         return self.find_nearest_station()
    
    def find_nearest_n_stations(self,n):
        if self.unit == 'm':
            distances = [(x[0], round(distance.distance(self.lat_lon, x[1]).miles,1)) for x in GB_STATIONS]
            distances.sort(key=lambda x:x[1])
            return (distances[:n])
        elif self.unit == 'km':
            distances = [(x[0], round(distance.distance(self.lat_lon, x[1]).km,1)) for x in GB_STATIONS]
            distances.sort(key=lambda x:x[1])
            return (distances[:n])

In [54]:
x = NearestStation('N4 4AF')

In [55]:
len(x.postcode)

6

In [52]:
x.geo_dict

{'postcode': 'N4 4AF', 'lat_lon': (51.573188, -0.10576)}

In [45]:
x.lat_lon

(51.573188, -0.10576)

In [14]:
x.endpoint

'http://api.postcodes.io/postcodes/n44af'

In [46]:
x.postcode

'n4 4af'

In [56]:
x.nearest_station

('Harringay', 0.3)

In [57]:
x.unit

'm'

In [58]:
x.unit ='km' #should change unit

In [59]:
x.unit

'km'

In [62]:
x.find_nearest_station()

('Harringay', 0.5)

In [61]:
x.nearest_station

('Harringay', 0.3)

In [ ]:
x.unit

In [ ]:
x.nearest_station #it does not - you need getters and setters!

In [ ]:
x.find_nearest_station('km')

In [ ]:
x.find_nearest_n_stations(10)

In [ ]:
x.find_nearest_n_stations(10)

In [ ]:
NearestStation('e17 3ss').nearest_station

In [ ]:
# Clean station data?
NearestStation('e17 3ss').find_nearest_n_stations(5)